In [6]:
import readlif
from readlif.reader import LifFile
import platform
import os
import numpy as np
import napari
import time
import tifffile

from napari.settings import get_settings
get_settings().application.ipy_interactive = True

In [8]:
# Inputs
save_path = r''
lif_path = r''

## FORMAT:   '[Channel]':'[Label]'
# If not using a channel, comment that line out. 
# When loading images from .lif, channels are taken in order... ensure used channels are ascending by wavelength in this list.
channels = {'DAPI': 'DAPI',         
            #'AF-488': '',
            #'AF-555': '',
            #'AF-647': '',
            #'AF-750': ''
            }

# FORMAT: 'Condition': ['List of Wells']
cols_key = {'': [],
            '': [],
            '': [],
            '': []}

In [9]:
if os.path.isfile(lif_path): print('File found.')
else: print('File not found.')

st = time.time()
lif_file = LifFile(lif_path)
print(f'File Loaded ({round(time.time()-st,2)}s).')

File found.
File Loaded (1.14s).


In [10]:
full_im_list = [x for x in lif_file.get_iter_image()]

slide_names = list(np.unique([x.name.split('/')[0] for x in full_im_list]))

In [ ]:
# list all images
[x.name for x in full_im_list]

In [12]:
# list individual slide names
slide_names

['oc3-cbpp', 'oc8-cbpp']

In [13]:
# List images in each slide
for slide in slide_names:
        print(f'Image Batch: {slide}')
        name_list = [x.name for x in full_im_list if slide in x.name]
        print([x.name for x in full_im_list if slide in x.name])
        print('\n')

Image Batch: oc3-cbpp
['oc3-cbpp/2_Merged', 'oc3-cbpp/5_Merged', 'oc3-cbpp/7_Merged', 'oc3-cbpp/10_Merged', 'oc3-cbpp/1_Merged', 'oc3-cbpp/6_Merged', 'oc3-cbpp/9_Merged', 'oc3-cbpp/10_Merged', 'oc3-cbpp/1_Merged', 'oc3-cbpp/4_Merged', 'oc3-cbpp/7_Merged', 'oc3-cbpp/10_Merged', 'oc3-cbpp/1_Merged', 'oc3-cbpp/4_Merged', 'oc3-cbpp/9_Merged', 'oc3-cbpp/12_Merged', 'oc3-cbpp/3_Merged', 'oc3-cbpp/6_Merged', 'oc3-cbpp/9_Merged', 'oc3-cbpp/12_Merged', 'oc3-cbpp/3_Merged', 'oc3-cbpp/6_Merged', 'oc3-cbpp/9_Merged', 'oc3-cbpp/12_Merged', 'oc3-cbpp/2_Merged', 'oc3-cbpp/6_Merged', 'oc3-cbpp/8_Merged', 'oc3-cbpp/12_Merged', 'oc3-cbpp/2_Merged', 'oc3-cbpp/6_Merged', 'oc3-cbpp/7_Merged', 'oc3-cbpp/12_Merged']


Image Batch: oc8-cbpp
['oc8-cbpp/2_Merged', 'oc8-cbpp/4_Merged', 'oc8-cbpp/7_Merged', 'oc8-cbpp/10_Merged', 'oc8-cbpp/1_Merged', 'oc8-cbpp/4_Merged', 'oc8-cbpp/7_Merged', 'oc8-cbpp/11_Merged', 'oc8-cbpp/3_Merged', 'oc8-cbpp/6_Merged', 'oc8-cbpp/9_Merged', 'oc8-cbpp/12_Merged', 'oc8-cbpp/3_Merge

In [14]:
## Input slides & conditions you would like to process. 

# Add any image batches you want to process here. Copy/paste from above output
slides_to_process = [] 

# Input conditions to process here. Copy/paste from cols_key
conditions_to_process = []

In [41]:
## Iterates through .lif files and saves images
# Execute cell with all inputs above complete

# Set up experiment name and directory
exp_name = lif_path.split('/')[-1].split('\\')[-1].replace('.lif','')
save_dir = os.path.join(save_path,exp_name)

# Make directory if not already made
if not os.path.isdir(save_dir): os.mkdir(save_dir)

# Iterate through slides_to_process
for slide in slides_to_process:
    print(slide)
    im_list = [x for x in full_im_list if slide in x.name and 'Merged' in x.name]

    # Make sub-directory storing slide images if not already made.
    slide_dir = os.path.join(save_dir,slide)
    if not os.path.isdir(slide_dir): os.mkdir(slide_dir)

    # Iterate through images
    for index,im in enumerate(im_list):
        print(f'\t{im.name}')

        ## If name has format of: {Slide Name}/{Row}{Col} 
        # Naming format occurs if experiment saved w/o 'raw data' folders
        if len(im.name.split('/')) == 2: 

            # Currently set up for 96-well plates-- if the first part of the name isn't a row
            if im.name.split(('/'))[0] not in ['A','B','C','D','E','F','G','H']:

                # Get column number & find associated condition
                col_pos = im.name.split('/')[-1].split('_')[0]
                condition = [x for x in cols_key if int(col_pos) in cols_key[x]][0]

                rows = ['A','B','C','D','E','F','G','H']

                ## IF ROW NOT IN NAME-- Find row identity and place in naming
                # Since THUNDER staves data by row, every 4th image belongs to the same row (Each of the 4 drugs has 1 image per concentation)
                iter_var = int(index)
                rownum = 0

                # Subtract index by 4 until negative; # of times subtracted(-1) indicates row
                while iter_var >= 0:    
                    iter_var = iter_var-4

                    if iter_var >0: rownum = rownum+1
                    else: break
                
                # Assign letter to row number
                curr_row = rows[rownum]
                plate_loc = '-'.join([curr_row,condition])

            else:
                plate_loc = im.name.split('/')[-1].split('_')[0]


        # If name has format of: {Slide Name}/{Row}/{Col}
        if len(im.name.split('/')) == 3:
            plate_loc = im.name.split('/')[-2:]
            num_pos = int(plate_loc[-1].split('_')[0])
            plate_loc[-1] = [x for x in cols_key if num_pos in cols_key[x]][0]
            plate_loc = '-'.join(plate_loc)

        ## Create save_name in format of slidename_row-drug.ome.tif
        # plate_loc in format of row-drug
        save_name = slide+'_'+plate_loc+'.ome.tif'
        print(f'\t\t\t',save_name)
        fin_save_path = os.path.join(slide_dir,save_name)

        # If image already saved, skip over it.
        if os.path.isfile(fin_save_path): print(f'\t\tAlready done.'); continue

        # Load all channels into an image stack.
        st = time.time() #timer start

        print(f'\t\tLoading Image')
        
        did_stack = False

        for ind,x in enumerate(im.get_iter_c()):
            image = np.array(x)

            if not did_stack:
                stacked_image = np.array(image)
                did_stack = True
            else:
                stacked_image = np.stack([stacked_image,image])
            

        print(f'\t\t\t{ind+1}/{im.channels} channels loaded.', end='\r')

        metadata = { 'axes': 'CYX', 'channels': channels }

        tifffile.imwrite(fin_save_path,
                         data = stacked_image,
                         metadata = metadata)
        
        print(f'\t\tStack loaded & saved. ({round(time.time(),2)-st} s)')

oc3-cbpp
	oc3-cbpp/2_Merged
			 oc3-cbpp_A-cvt.ome.tif
		Loading Image
		Stack loaded & saved. (52.03377866744995 s)
	oc3-cbpp/5_Merged
			 oc3-cbpp_A-berzosertib.ome.tif
		Loading Image
		Stack loaded & saved. (41.71278405189514 s)
	oc3-cbpp/7_Merged
			 oc3-cbpp_A-palbociclib.ome.tif
		Loading Image
		Stack loaded & saved. (40.33141613006592 s)
	oc3-cbpp/10_Merged
			 oc3-cbpp_A-pf.ome.tif
		Loading Image
		Stack loaded & saved. (32.28991150856018 s)
	oc3-cbpp/1_Merged
			 oc3-cbpp_A-cvt.ome.tif
		Already done.
	oc3-cbpp/6_Merged
			 oc3-cbpp_B-berzosertib.ome.tif
		Loading Image
		Stack loaded & saved. (37.181925773620605 s)
	oc3-cbpp/9_Merged
			 oc3-cbpp_B-palbociclib.ome.tif
		Loading Image
		Stack loaded & saved. (36.00437331199646 s)
	oc3-cbpp/10_Merged
			 oc3-cbpp_B-pf.ome.tif
		Loading Image
		Stack loaded & saved. (43.394697189331055 s)
	oc3-cbpp/1_Merged
			 oc3-cbpp_B-cvt.ome.tif
		Loading Image
		Stack loaded & saved. (33.08952832221985 s)
	oc3-cbpp/4_Merged
			 oc3-cbpp